Bibliotecas

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import os
from PIL import Image

In [ ]:
def find_parent_contours(bordas, min_contour_area):
    contours, _ = cv2.findContours(bordas, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    parent_contours = [contour for contour in contours if cv2.contourArea(contour) > min_contour_area]
    parent_contours_df = pd.DataFrame({'contour': parent_contours})
    return parent_contours_df

def generate_random_lines(num_lines, max_line_length, max_angle_deviation, max_rho_deviation):
    lines = []
    for _ in range(num_lines):
        rho = random.uniform(0, np.sqrt(imagem_filtrada.shape[0]**2 + imagem_filtrada.shape[1]**2))
        theta = random.uniform(0, np.pi)
        lines.append((rho, theta))
    return add_randomness_to_lines(lines, max_angle_deviation, max_rho_deviation)

def add_randomness_to_lines(lines, max_angle_deviation, max_rho_deviation):
    modified_lines = []
    for line in lines:
        rho, theta = line
        modified_theta = theta + random.uniform(-max_angle_deviation, max_angle_deviation)
        modified_rho = rho + random.uniform(-max_rho_deviation, max_rho_deviation)
        modified_lines.append((modified_rho, modified_theta))
    return modified_lines

def check_points_in_lines(contour_points, lines, tolerance):
    pontos_contornos = []

    for point in contour_points:
        x, y = point[0]
        point_found = False

        for rho, theta in lines:
            a = np.cos(theta)
            b = np.sin(theta)
            dist = abs(a * x + b * y - rho) / np.sqrt(a**2 + b**2)

            if dist < tolerance:
                point_found = True
                break

        if point_found:
            pontos_contornos.append((x, y))

    return pontos_contornos

def check_point_in_line(point, rho, theta, tolerance):
    x, y = point
    a = np.cos(theta)
    b = np.sin(theta)
    dist = abs(a * x + b * y - rho) / np.sqrt(a**2 + b**2)
    return dist < tolerance

def fator_conversao(regua_micro, regua_macro, resolucao_ppi):
    conversao_micrometros = (regua_micro / regua_macro )* 2.54 *(1 / resolucao_ppi)
    return conversao_micrometros

Carregamento da Imagem

In [ ]:
def find_parent_contours(bordas, min_contour_area):
    contours, _ = cv2.findContours(bordas, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    parent_contours = [contour for contour in contours if cv2.contourArea(contour) > min_contour_area]
    parent_contours_df = pd.DataFrame({'contour': parent_contours})
    return parent_contours_df

def generate_random_lines(num_lines, max_line_length, max_angle_deviation, max_rho_deviation):
    lines = []
    for _ in range(num_lines):
        rho = random.uniform(0, np.sqrt(imagem_filtrada.shape[0]**2 + imagem_filtrada.shape[1]**2))
        theta = random.uniform(0, np.pi)
        lines.append((rho, theta))
    return add_randomness_to_lines(lines, max_angle_deviation, max_rho_deviation)

def add_randomness_to_lines(lines, max_angle_deviation, max_rho_deviation):
    modified_lines = []
    for line in lines:
        rho, theta = line
        modified_theta = theta + random.uniform(-max_angle_deviation, max_angle_deviation)
        modified_rho = rho + random.uniform(-max_rho_deviation, max_rho_deviation)
        modified_lines.append((modified_rho, modified_theta))
    return modified_lines

def check_points_in_lines(contour_points, lines, tolerance):
    pontos_contornos = []

    for point in contour_points:
        x, y = point[0]
        point_found = False

        for rho, theta in lines:
            a = np.cos(theta)
            b = np.sin(theta)
            dist = abs(a * x + b * y - rho) / np.sqrt(a**2 + b**2)

            if dist < tolerance:
                point_found = True
                break

        if point_found:
            pontos_contornos.append((x, y))

    return pontos_contornos

def check_point_in_line(point, rho, theta, tolerance):
    x, y = point
    a = np.cos(theta)
    b = np.sin(theta)
    dist = abs(a * x + b * y - rho) / np.sqrt(a**2 + b**2)
    return dist < tolerance

def fator_conversao(regua_micro, regua_macro, resolucao_ppi):
    conversao_micrometros = (regua_micro / regua_macro )* 2.54 *(1 / resolucao_ppi)
    return conversao_micrometros

Aplicação do Método de Canny

In [ ]:
# Aplicar o método de Canny para detecção de bordas
bordas = cv2.Canny(imagem_filtrada, 50, 50)

Visualização das Bordas

In [ ]:
# Exibir a imagem com as bordas detectadas
plt.imshow(bordas, cmap='gray')
plt.title('Bordas detectadas com o método de Canny')
plt.axis('off')
plt.show()

Criação das Retas

In [ ]:
try:
    if imagem_filtrada is None:
        raise ValueError("Não foi possível ler a imagem. Verifique o caminho da imagem.")

    # Encontrar os contornos relevantes na imagem
    parent_contours_df = find_parent_contours(bordas, min_contour_area = 10 )

    if len(parent_contours_df) >= 2:
        # Gerar linhas aleatórias
        max_random_lines = 40
        max_line_length = 500
        max_angle_deviation = np.pi / 2
        max_rho_deviation = 100  # Pixels
        randomized_lines = generate_random_lines(max_random_lines, max_line_length, max_angle_deviation, max_rho_deviation)

        # Desenhar as linhas detectadas com aleatoriedade
        for rho, theta in randomized_lines:
            a = np.cos(theta)
            b = np.sin(theta)
            x0 = a * rho
            y0 = b * rho
            x1 = int(x0 + max_line_length * (-b))
            y1 = int(y0 + max_line_length * (a))
            x2 = int(x0 - max_line_length * (-b))
            y2 = int(y0 - max_line_length * (a))
            cv2.line(imagem_filtrada, (x1, y1), (x2, y2), (0, 0, 255), 2)

        # Verificar se os pontos dos contornos estão contidos nas retas
        tolerance = 5

        for _, row in parent_contours_df.iterrows():
            contour_points = row['contour']
            check_points_in_lines(contour_points, randomized_lines, tolerance)

    else:
        print("Não foram encontrados contornos suficientes na imagem.")

except ValueError as ve:
    print("Erro ao processar a imagem:", ve)
except Exception as e:
    print("Ocorreu um erro inesperado:", e)

Visualização das Imagens com as retas

In [ ]:
# Exibir a imagem com as linhas traçadas
cv2_imshow(imagem_filtrada)
cv2.waitKey(0)
cv2.destroyAllWindows()